In [1]:
import altair as alt
import pandas as pd

import gif

In [8]:
df = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/full_data.csv')
df['date'] = pd.to_datetime(df['date'])

In [96]:
select = df[df['location'].isin([
    'United States',
    'China',
    'Canada',
    'Sweden']
)].reset_index(drop=True)

In [97]:
population = {
    'United States': 330_000_000,
    'China': 1_440_000_000,
    'Canada': 37_000_000,
    'Sweden': 10_000_100
}

In [98]:
select['population'] = select['location'].map(population)
select['cases_per_million'] = select.apply(lambda row: row['new_cases'] / row['population'] * 1_000_000, axis=1)
select['7d'] = select.groupby('location').rolling(7)['cases_per_million'].mean().reset_index(drop=True)

In [108]:
@gif.frame
def plot(date):
    d = select[select['date'] <= date]
    
    chart = alt.Chart(d).encode(
        x=alt.X('date', title=None),
        y=alt.Y('7d', title=None),
        color=alt.Color('location', 
            scale=alt.Scale(
                domain=['United States', 'China', 'Canada', 'Sweden'], 
                range=["#000055", "#550000", "#881111", "#2222CC"]
            ),
            legend=None
        )
    ).mark_line()
    
    return chart

In [ ]:
dates = pd.date_range(start='2020-02-01', end=df['date'].max())
frames = []
for date in dates:
    frame = plot(date)
    frames.append(frame)

gif.save(frames, 'covid.gif', duration=50)